In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import Dataset,DataLoader,random_split

torch.manual_seed(666)
torch.cuda.manual_seed(666)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

## 数据集：多种肺炎X光图像

### 数据集

In [2]:
data_dir = "../data/COVID_Dataset"
TRAIN = 'train'
TEST = 'test'
VAL = 'val'

def apply_transform(mode=None):
    size = (299,299)
    crop = 299
    if mode == 'train':
        transform = transforms.Compose([transforms.Resize(size),
                               transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation((-20,+20)),
                               transforms.CenterCrop(crop),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                              ])

    elif mode == 'test' or mode == 'val':
        transform = transforms.Compose([transforms.Resize(size),
                               transforms.CenterCrop(crop),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                              ])

    return transform

trainset = datasets.ImageFolder(os.path.join(data_dir, TRAIN),
                                transform = apply_transform(TRAIN))

trainset,valset = random_split(trainset,[len(trainset)-int(len(trainset)*0.1),int(len(trainset)*0.1)])

testset = datasets.ImageFolder(os.path.join(data_dir, TEST),
                               transform = apply_transform(TEST))

train_loader = DataLoader(trainset,
                          batch_size=50,
                          shuffle=True)

val_loader = DataLoader(valset,
                        batch_size=10)

test_loader = DataLoader(testset,
                         batch_size=1)

len(train_loader),len(test_loader),len(val_loader)




(439, 2706, 244)

### 模型

In [3]:
# Inception 残差块
class Residual(nn.Module):
    def __init__(self,in_channels,out_channels,inception,kernel_size=(1,1),stride=1,padding=0) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=kernel_size,stride=stride,padding=padding)
        self.inception = inception
    def forward(self,x):
        conv = self.conv1(x)
        y = self.inception(x)
        out = F.relu(conv+y)
        return out

def get_deepLavV3(out_ch):
    model = torchvision.models.inception_v3(pretrained=True)
    # 在每个Inception层 添加残差
    model.Mixed_5b = Residual(192,256,model.Mixed_5b)
    model.Mixed_5c = Residual(256,288,model.Mixed_5c)
    model.Mixed_5d = Residual(288,288,model.Mixed_5d)
    model.Mixed_6a = Residual(288,768,model.Mixed_6a,(5,5),2,1)
    model.Mixed_6b = Residual(768,768,model.Mixed_6b)
    model.Mixed_6c = Residual(768,768,model.Mixed_6c)
    model.Mixed_6d = Residual(768,768,model.Mixed_6d)
    model.Mixed_6e = Residual(768,768,model.Mixed_6e)
    model.Mixed_7a = Residual(768,1280,model.Mixed_7a,(5,5),2,1)
    model.Mixed_7b = Residual(1280,2048,model.Mixed_7b)
    model.Mixed_7c = Residual(2048,2048,model.Mixed_7c)

    # 冻结参数
    for param in model.parameters():
        param.requires_grad = False
    # 改全连接层
    in_features = model.fc.in_features
    classifier = nn.Sequential(
        nn.Linear(in_features, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, out_ch),
        nn.LogSoftmax(dim=1)
    )
    model.fc = classifier
    return model

# model = get_deepLavV3(2)
# print(model)
# x = torch.tensor(np.random.random((2,3,299,299))).float()
# y,_ = model(x)
# y.shape,_.shape


### 评价指标

In [4]:
def accuracy(preds, labels):
    preds = torch.exp(preds)
    top_p,top_class = preds.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

### 冻结训练

In [5]:
model = get_deepLavV3(4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

epochs = 25
val_loss_min = np.Inf
max_e = 10

# 模型存放路径
model_path = os.path.join('./model')
name = "Res_InceptionV3_pre"
model = model.to(DEVICE)
for epoch in range(epochs):

    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0.0
    val_acc = 0.0

    model.train()
    for images,labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        preds,_ = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(preds, labels)

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)

    model.eval()
    with torch.no_grad():
        for images,labels in tqdm(val_loader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            loss = criterion(preds, labels)
            val_loss += loss.item()
            val_acc += accuracy(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

    schedular.step(avg_val_loss)

    print("Epoch : {} \ntrain_loss : {:.6f}, \tTrain_acc : {:.6f}, \nVal_loss : {:.6f}, \tVal_acc : {:.6f}".format(epoch + 1,
                                                                                                                   avg_train_loss, avg_train_acc,
                                                                                                                   avg_val_loss, avg_val_acc))
    if avg_val_loss <= val_loss_min:
        print('Validation loss decreased from ({:.6f} --> {:.6f}).\nSaving model ...'.format(val_loss_min, avg_val_loss))
        torch.save(model,os.path.join(model_path,f"{name}.pth"))
        val_loss_min = avg_val_loss
        max_e = 20
    max_e -= 1
    if max_e<=0:
        break


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/featurize/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

100%|██████████| 244/244 [00:24<00:00,  9.90it/s]


Epoch : 1 
train_loss : 1.130528, 	Train_acc : 0.519214, 
Val_loss : 0.981015, 	Val_acc : 0.617213
Validation loss decreased from (inf --> 0.981015).
Saving model ...


100%|██████████| 244/244 [00:24<00:00, 10.05it/s]


...

Epoch : 23 
train_loss : 0.942678, 	Train_acc : 0.598603, 
Val_loss : 0.901195, 	Val_acc : 0.687295


100%|██████████| 244/244 [00:23<00:00, 10.59it/s]


Epoch : 24 
train_loss : 0.944298, 	Train_acc : 0.599398, 
Val_loss : 0.900669, 	Val_acc : 0.677049


100%|██████████| 244/244 [00:23<00:00, 10.49it/s]

Epoch : 25 
train_loss : 0.948375, 	Train_acc : 0.601128, 
Val_loss : 0.901749, 	Val_acc : 0.683607


In [6]:
val_loss_min

0.8903199996127457

### 解冻参数

In [7]:
model = torch.load(os.path.join(model_path,f"{name}.pth"))

for param in model.parameters():
        param.requires_grad = True

# for k,v in model.named_parameters():
#     print(f"{k}: {v.requires_grad}")
# model

### finetune

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

epochs =50
# val_loss_min = np.Inf
max_e = 10

name = "Res_InceptionV3_pre"
model = model.to(DEVICE)
for epoch in range(epochs):

    train_loss = 0.0
    val_loss = 0.0
    train_acc = 0.0
    val_acc = 0.0

    model.train()
    for images,labels in tqdm(train_loader):
        optimizer.zero_grad()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds,_ = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(preds, labels)

    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)

    model.eval()
    with torch.no_grad():
        for images,labels in tqdm(val_loader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            loss = criterion(preds, labels)
            val_loss += loss.item()
            val_acc += accuracy(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

    schedular.step(avg_val_loss)

    print("Epoch : {} \ntrain_loss : {:.6f}, \tTrain_acc : {:.6f}, \nVal_loss : {:.6f}, \tVal_acc : {:.6f}".format(epoch + 1,
                                                                                                                   avg_train_loss, avg_train_acc,
                                                                                                                   avg_val_loss, avg_val_acc))
    if avg_val_loss <= val_loss_min:
        print('Validation loss decreased from ({:.6f} --> {:.6f}).\nSaving model ...'.format(val_loss_min, avg_val_loss))
        torch.save(model, os.path.join(model_path,f"{name}.pth"))
        val_loss_min = avg_val_loss
        max_e = 10
    max_e -= 1
    if max_e<=0:
        break


100%|██████████| 244/244 [00:23<00:00, 10.59it/s]


Epoch : 1 
train_loss : 0.657715, 	Train_acc : 0.741113, 
Val_loss : 0.386418, 	Val_acc : 0.873224
Validation loss decreased from (0.890320 --> 0.386418).
Saving model ...


100%|██████████| 244/244 [00:23<00:00, 10.40it/s]


Epoch : 2 
train_loss : 0.392956, 	Train_acc : 0.857103, 
Val_loss : 0.372892, 	Val_acc : 0.864208
Validation loss decreased from (0.386418 --> 0.372892).
Saving model ...


100%|██████████| 244/244 [00:24<00:00, 10.15it/s]


...

Epoch : 21 
train_loss : 0.066138, 	Train_acc : 0.975535, 
Val_loss : 0.177611, 	Val_acc : 0.938524


100%|██████████| 244/244 [00:23<00:00, 10.24it/s]


Epoch : 22 
train_loss : 0.063385, 	Train_acc : 0.977176, 
Val_loss : 0.180579, 	Val_acc : 0.933606


100%|██████████| 244/244 [00:23<00:00, 10.42it/s]

Epoch : 23 
train_loss : 0.062125, 	Train_acc : 0.977314, 
Val_loss : 0.179738, 	Val_acc : 0.937704


In [9]:
val_loss_min

0.16859943654195436

### 测试

In [7]:
name = "Res_InceptionV3_pre"
model_path = os.path.join('./model')
model = torch.load(os.path.join(model_path,f"{name}.pth"))
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

# 混淆矩阵
M = np.zeros((4,4))
with torch.no_grad():
    test_loss=0
    test_acc=0
    for images,labels in tqdm(test_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        preds = model(images)
        loss = criterion(preds, labels)
        test_loss += loss.item()
        test_acc += accuracy(preds, labels)
        
        M[preds.argmax(),labels[0]]+=1
        
    avg_test_loss = test_loss / len(test_loader)
    avg_test_acc = test_acc / len(test_loader)
    print("Test_loss : {:.6f}, \tTest_acc : {:.6f}".format(avg_test_loss,avg_test_acc))
    
    M = M.astype("int")
    print(M)
    

100%|██████████| 2706/2706 [01:10<00:00, 38.60it/s]

Test_loss : 0.160139, 	Test_acc : 0.942720
[[ 362    4    2    0]
 [   1  798   29   28]
 [   3   35 1137    2]
 [   1   42    8  254]]
